<div class='heading'>
    <div style='float:left;'><h1>CPSC 4300/6300: Applied Data Science</h1></div>
    <img style="float: right; padding-right: 10px; width: 65px" src="https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/images/clemson_paw.png"> </div>


## Week 2: Introduction to Pandas

**Clemson University**<br>
**Fall 2021**<br>
**Instructor(s):** Nina Hubig <br>
**Author(s):** Brandon Walker


---

In [ ]:
""" RUN THIS CELL TO GET THE RIGHT FORMATTING """
import requests
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/css/cpsc6300.css'
styles = requests.get(css_file).text
HTML(styles)

# Table of Contents
<ol start="0">
<li> Learning Goals </li>
<li> Loading and Cleaning with Pandas</li>
<li> Parsing and Completing the Dataframe  </li>
<li> Grouping </li>
</ol>

## Learning Goals

About 6,000 odd "best books" were fetched and parsed from [Goodreads](https://www.goodreads.com). The "bestness" of these books came from a proprietary formula used by Goodreads and published as a list on their web site.

We parsed the page for each book and saved data from all these pages in a tabular format as a CSV file. In this lab we'll clean and further parse the data.  We'll then do some exploratory data analysis to answer questions about these best books and popular genres.  


By the end of this lab, you should be able to:

- Load and systematically address missing values, encoded as `NaN` values in our data set, for example, by removing observations associated with these values.
- Parse columns in the dataframe to create new dataframe columns.
- Use groupby to aggregate data on a particular feature column, such as author.

### Basic EDA workflow

The basic workflow is as follows:

1. **Build** a DataFrame from the data (ideally, put all data in this object)
2. **Clean** the DataFrame. It should have the following properties:
    - Each row describes a single object
    - Each column describes a property of that object
    - Columns are numeric whenever appropriate
    - Columns contain atomic properties that cannot be further decomposed
3. Explore **global properties**. Use histograms, scatter plots, and aggregation functions to summarize the data.
4. Explore **group properties**. Use groupby and small multiples to compare subsets of the data.

This process transforms your data into a format which is easier to work with, gives you a basic overview of the data's properties, and likely generates several questions for you to followup in subsequent analysis.

## Part 1: Loading and Cleaning with Pandas
Read in the `goodreads.csv` file, examine the data, and do any necessary data cleaning.

Here is a description of the columns (in order) present in this csv file:

```
rating: the average rating on a 1-5 scale achieved by the book
review_count: the number of Goodreads users who reviewed this book
isbn: the ISBN code for the book
booktype: an internal Goodreads identifier for the book
author_url: the Goodreads (relative) URL for the author of the book
year: the year the book was published
genre_urls: a string with '|' separated relative URLS of Goodreads genre pages
dir: a directory identifier internal to the scraping code
rating_count: the number of ratings for this book (this is different from the number of reviews)
name: the name of the book
```

Let us see what issues we find with the data and resolve them.  



----




After loading appropriate libraries


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

### Cleaning: Reading in the data
We read in and clean the data from `goodreads.csv`.

In [ ]:
#Read the data into a dataframe
df = pd.read_csv("goodreads.csv", encoding='utf-8')

#Examine the first few rows of the dataframe
df

,4.40,136455,0439023483,good_reads:book,https://www.goodreads.com/author/show/153394.Suzanne_Collins,2008,/genres/young-adult|/genres/science-fiction|/genres/dystopia|/genres/fantasy|/genres/science-fiction|/genres/romance|/genres/adventure|/genres/book-club|/genres/young-adult|/genres/teen|/genres/apocalyptic|/genres/post-apocalyptic|/genres/action,dir01/2767052-the-hunger-games.html,2958974,"The Hunger Games (The Hunger Games, #1)"
0,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003.0,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...
1,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005.0,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)"
2,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960.0,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird
3,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813.0,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice
4,4.25,12363,0446675539,good_reads:book,https://www.goodreads.com/author/show/11081.Ma...,1936.0,/genres/classics|/genres/historical-fiction|/g...,dir01/18405.Gone_with_the_Wind.html,645470,Gone with the Wind
...,...,...,...,...,...,...,...,...,...,...
5994,4.17,2226,0767913736,good_reads:book,https://www.goodreads.com/author/show/44565.Ca...,2005.0,/genres/history|/genres/non-fiction|/genres/bi...,dir60/78508.The_River_of_Doubt.html,16618,The River of Doubt
5995,3.99,775,1416909427,good_reads:book,https://www.goodreads.com/author/show/151371.J...,2006.0,/genres/young-adult|/genres/realistic-fiction|...,dir60/259068.Shug.html,6179,Shug
5996,3.78,540,1620612321,good_reads:book,https://www.goodreads.com/author/show/5761314....,2012.0,/genres/contemporary|/genres/romance|/genres/y...,dir60/13503247-flawed.html,2971,Flawed
5997,3.91,281,NaN,good_reads:book,https://www.goodreads.com/author/show/1201952....,2006.0,/genres/religion|/genres/islam|/genres/religio...,dir60/2750008.html,3083,Ø£Ø³Ø¹Ø¯ Ø§Ù Ø±Ø£Ø© ÙÙ Ø§ÙØ¹Ø§ÙÙ


Oh dear. That does not quite seem to be right. We are missing the column names. We need to add these in! But what are they?

Here is a list of them in order:

`["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name']`

<div class="exercise"><b>Exercise</b></div>
Use these to load the dataframe properly! And then "head" the dataframe... (you will need to look at the read_csv docs)


In [ ]:
# your code here
df.columns=["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name']
df
#Examine the first few rows of the dataframe
df.head(10)


,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
0,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003.0,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...
1,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005.0,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)"
2,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960.0,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird
3,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813.0,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice
4,4.25,12363,0446675539,good_reads:book,https://www.goodreads.com/author/show/11081.Ma...,1936.0,/genres/classics|/genres/historical-fiction|/g...,dir01/18405.Gone_with_the_Wind.html,645470,Gone with the Wind
5,4.22,7205,0066238501,good_reads:book,https://www.goodreads.com/author/show/1069006....,1949.0,/genres/classics|/genres/young-adult|/genres/c...,dir01/11127.The_Chronicles_of_Narnia.html,286677,The Chronicles of Narnia (Chronicles of Narnia...
6,4.38,10902,0060256656,good_reads:book,https://www.goodreads.com/author/show/435477.S...,1964.0,/genres/childrens|/genres/young-adult|/genres/...,dir01/370493.The_Giving_Tree.html,502891,The Giving Tree
7,3.79,20670,0452284244,good_reads:book,https://www.goodreads.com/author/show/3706.Geo...,1945.0,/genres/classics|/genres/fiction|/genres/scien...,dir01/7613.Animal_Farm.html,1364879,Animal Farm
8,4.18,12302,0345391802,good_reads:book,https://www.goodreads.com/author/show/4.Dougla...,1979.0,/genres/science-fiction|/genres/humor|/genres/...,dir01/11.The_Hitchhiker_s_Guide_to_the_Galaxy....,724713,The Hitchhiker's Guide to the Galaxy (Hitchhik...
9,4.03,20937,0739326228,good_reads:book,https://www.goodreads.com/author/show/614.Arth...,1997.0,/genres/fiction|/genres/historical-fiction|/ge...,dir01/930.Memoirs_of_a_Geisha.html,1042679,Memoirs of a Geisha


### Cleaning: Examing the dataframe - quick checks

We should examine the dataframe to get a overall sense of the content.

<div class="exercise"><b>Exercise</b></div>
Lets check the types of the columns. What do you find?

In [ ]:
# your code here
dataType=df.dtypes
dataType

rating          float64
review_count     object
isbn             object
booktype         object
author_url       object
year            float64
genre_urls       object
dir              object
rating_count     object
name             object
dtype: object

*your answer here*

Notice that `review_count` and `rating_counts` are objects instead of ints, and the `year` is a float!

There are a couple more quick sanity checks to perform on the dataframe.

In [ ]:
print(df.shape)
df.columns

(5999, 10)


Index(['rating', 'review_count', 'isbn', 'booktype', 'author_url', 'year', 'genre_urls', 'dir', 'rating_count', 'name'], dtype='object')

### Cleaning: Examining the dataframe - a deeper look

Beyond performing checking some quick general properties of the data frame and looking at the first $n$ rows, we can dig a bit deeper into the values being stored. If you haven't already, check to see if there are any missing values in the data frame.

Let's see for a column which seemed OK to us.

In [ ]:
#Get a sense of how many missing values there are in the dataframe.
print(np.sum([df.rating.isnull()]))
print(np.sum([df.review_count.isnull()]))
print(np.sum([df.isbn.isnull()]))
print(np.sum([df.booktype.isnull()]))
print(np.sum([df.author_url.isnull()]))
print(np.sum([df.year.isnull()]))
print(np.sum([df.genre_urls.isnull()]))
print(np.sum([df.dir.isnull()]))
print(np.sum([df.rating_count.isnull()]))
print(np.sum([df.name.isnull()]))

2
0
475
0
0
7
62
0
0
0


In [ ]:
#Try to locate where the missing values occur
df[df.rating.isnull()]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
3642,NaN,None,None,None,None,NaN,NaN,dir37/9658936-harry-potter.html,None,None
5281,NaN,None,None,None,None,NaN,NaN,dir53/113138.The_Winner.html,None,None


How does `pandas` or `numpy` handle missing values when we try to compute with data sets that include them?

We'll now check if any of the other suspicious columns have missing values.  Let's look at `year` and `review_count` first.

One thing you can do is to try and convert to the type you expect the column to be. If something goes wrong, it likely means your data are bad.

Lets test for missing data:

In [ ]:
df[df.year.isnull()]

df.year.isnull()
df.shape

(5999, 10)

### Cleaning: Dealing with Missing Values
How should we interpret 'missing' or 'invalid' values in the data (hint: look at where these values occur)? One approach is to simply exclude them from the dataframe. Is this appropriate for all 'missing' or 'invalid' values?

In [ ]:
#Treat the missing or invalid values in your dataframe
#######

df = df[df.year.notnull()]

Ok so we have done some cleaning. What do things look like now? Notice the float has not yet changed.

In [ ]:
df.dtypes

rating          float64
review_count     object
isbn             object
booktype         object
author_url       object
year            float64
genre_urls       object
dir              object
rating_count     object
name             object
dtype: object

In [ ]:
print(np.sum(df.year.isnull()))
df.shape # We removed seven rows

0


(5992, 10)

<div class="exercise"><b>Exercise</b></div>

Ok so lets fix those types. Convert them to ints. If the type conversion fails, we now know we have further problems.

In [ ]:
# your code here
df.year = df.year.astype(int)
df.review_count = df.review_count.astype(int)
df.rating_count = df.rating_count.astype(int)

/var/folders/lf/vb5zc99j7sx183h5f_gy9djr0000gn/T/ipykernel_30565/3297262744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.year = df.year.astype(int)
/var/folders/lf/vb5zc99j7sx183h5f_gy9djr0000gn/T/ipykernel_30565/3297262744.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.review_count = df.review_count.astype(int)
/var/folders/lf/vb5zc99j7sx183h5f_gy9djr0000gn/T/ipykernel_30565/3297262744.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

Once you do this, we seem to be good on these columns (no errors in conversion). Lets look:

In [ ]:
df.dtypes

rating          float64
review_count      int64
isbn             object
booktype         object
author_url       object
year              int64
genre_urls       object
dir              object
rating_count      int64
name             object
dtype: object

Sweet!

Some of the other colums that should be strings have NaN.

In [ ]:
df.loc[df.genre_urls.isnull(), 'genre_urls']=""
df.loc[df.isbn.isnull(), 'isbn']=""

##  Part 2: Parsing and Completing the Data Frame

We will parse the `author` column from the author_url and `genres` column from the genre_urls. Keep the `genres` column as a string separated by '|'.

We will use panda's `map` to assign new columns to the dataframe.  

Examine an example `author_url` and reason about which sequence of string operations must be performed in order to isolate the author's name.

In [ ]:
#Get the first author_url
test_string = df.author_url[0]
test_string

'https://www.goodreads.com/author/show/1077326.J_K_Rowling'

In [ ]:
#Test out some string operations to isolate the author name

test_string.split('/')[-1].split('.')[1:][0]

'J_K_Rowling'

<div class="exercise"><b>Exercise</b></div>

Lets wrap the above code into a function which we will then use

In [ ]:
# Write a function that accepts an author url and returns the author's name based on your experimentation above
def get_author(url):
    # your code here
    name = url.split('/')[-1].split('.')[1:][0]

    return name

In [ ]:
#Apply the get_author function to the 'author_url' column using '.map'
#and add a new column 'author' to store the names
df['author'] = df.author_url.map(get_author)
df.author[0:5]

/var/folders/lf/vb5zc99j7sx183h5f_gy9djr0000gn/T/ipykernel_30565/3494315277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['author'] = df.author_url.map(get_author)


0          J_K_Rowling
1      Stephenie_Meyer
2           Harper_Lee
3          Jane_Austen
4    Margaret_Mitchell
Name: author, dtype: object


Now parse out the genres from `genre_url`.  

This is a little more complicated because there be more than one genre.


In [ ]:

df.genre_urls.head()

0    /genres/fantasy|/genres/young-adult|/genres/fi...
1    /genres/young-adult|/genres/fantasy|/genres/ro...
2    /genres/classics|/genres/fiction|/genres/histo...
3    /genres/classics|/genres/fiction|/genres/roman...
4    /genres/classics|/genres/historical-fiction|/g...
Name: genre_urls, dtype: object

In [ ]:
# your code here
#Examine some examples of genre_urls

#Test out some string operations to isolate the genre name
test_genre_string=df.genre_urls[0]
genres=test_genre_string.strip().split('|')
for e in genres:
    print(e.split('/')[-1])
    "|".join(genres)

fantasy
young-adult
fiction
fantasy
magic
childrens
adventure
science-fiction-fantasy


<div class="exercise"><b>Exercise</b></div>

Write a function that accepts a genre url and returns the genre name based on your experimentation above



In [ ]:
def split_and_join_genres(url):
    # your code here
    name=url.strip().split('|')
    for e in name:
        print(e.split('/')[-1])
        "|".join(name)




Test your function

In [ ]:
split_and_join_genres("/genres/young-adult|/genres/science-fiction")

young-adult
science-fiction


In [ ]:
split_and_join_genres("")


Use map again to create a new "genres" column

In [ ]:

df['genres']=df.genre_urls.map(split_and_join_genres)
df.head()

fantasy
young-adult
fiction
fantasy
magic
childrens
adventure
science-fiction-fantasy
young-adult
fantasy
romance
paranormal
vampires
fiction
fantasy
paranormal
romance
paranormal-romance
fantasy
supernatural
young-adult
teen
fantasy
urban-fantasy
classics
fiction
historical-fiction
academic
school
literature
young-adult
academic
read-for-school
novels
book-club
young-adult
high-school
classics
fiction
romance
historical-fiction
literature
novels
european-literature
british-literature
classics
classic-literature
adult
academic
school
classics
historical-fiction
fiction
romance
literature
romance
historical-romance
war
military-history
civil-war
classics
young-adult
childrens
christian
adventure
science-fiction-fantasy
fantasy
religion
fantasy
magic
literature
childrens
young-adult
childrens
picture-books
fantasy
inspirational
classics
academic
school
philosophy
childrens
juvenile
short-stories
classics
fiction
science-fiction
dystopia
literature
fantasy
academic
school
politics
science

/var/folders/lf/vb5zc99j7sx183h5f_gy9djr0000gn/T/ipykernel_30565/2336828503.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres']=df.genre_urls.map(split_and_join_genres)


,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name,author,genres
0,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...,J_K_Rowling,None
1,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)",Stephenie_Meyer,None
2,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird,Harper_Lee,None
3,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice,Jane_Austen,None
4,4.25,12363,0446675539,good_reads:book,https://www.goodreads.com/author/show/11081.Ma...,1936,/genres/classics|/genres/historical-fiction|/g...,dir01/18405.Gone_with_the_Wind.html,645470,Gone with the Wind,Margaret_Mitchell,None


Finally, let's pick an author at random so we can see the results of the transformations.  Scroll to see the `author` and `genre` columns that we added to the dataframe.

In [ ]:
df[df.author == "Marguerite_Yourcenar"]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name,author,genres
1013,4.23,483,0374529264,good_reads:book,https://www.goodreads.com/author/show/7732.Mar...,1951,/genres/historical-fiction|/genres/fiction|/ge...,dir11/12172.Memoirs_of_Hadrian.html,6258,Memoirs of Hadrian,Marguerite_Yourcenar,None
5619,4.11,74,2070367983,good_reads:book,https://www.goodreads.com/author/show/7732.Mar...,1968,/genres/fiction|/genres/historical-fiction|/ge...,dir57/953435.L_uvre_au_noir.html,1601,L'Åuvre au noir,Marguerite_Yourcenar,None


Let us delete the `genre_urls` column.

In [ ]:
del df['genre_urls']

And then save the dataframe out!

In [ ]:
df.to_csv("data/cleaned-goodreads.csv", index=False, header=True)

OSError: Cannot save file into a non-existent directory: 'data'

---

## Part 3: Grouping

It appears that some books were written in negative years!  Print out the observations that correspond to negative years.  What do you notice about these books?  

In [ ]:
# your code here
df[df.year < 0].name
#These are books written before the Common Era (BCE, equivalent to BC).

46                               The Odyssey
245                    The Iliad/The Odyssey
454                             The Republic
595                               The Aeneid
628                              Oedipus Rex
673                           The Art of War
745                        The Bhagavad Gita
776                                 Antigone
1232                       The Oedipus Cycle
1396                          Aesop's Fables
1397                   The Epic of Gilgamesh
1427                                   Medea
1814                            The Oresteia
1881         The Trial and Death of Socrates
2077    The History of the Peloponnesian War
2526                           The Histories
3132                          Complete Works
3273                  The Nicomachean Ethics
3756                              Lysistrata
4401                           The Symposium
4474                                 Apology
5366                          Five Dialogues
Name: name

We can determine the "best book" by year! For this we use Panda's `groupby()`. `Groupby()` allows grouping a dataframe by any (usually categorical) variable. Would it make sense to ever groupby integer variables? Floating point variables?

In [ ]:
dfgb_author = df.groupby('author')
type(dfgb_author)

pandas.core.groupby.generic.DataFrameGroupBy

Perhaps we want the number of books each author wrote

In [ ]:
dfgb_author.count()

,rating,review_count,isbn,booktype,author_url,year,dir,rating_count,name,genres
author,,,,,,,,,,
A_A_Milne,6,6,6,6,6,6,6,6,6,0
A_G_Howard,1,1,1,1,1,1,1,1,1,0
A_J_Cronin,1,1,1,1,1,1,1,1,1,0
A_J_Jacobs,1,1,1,1,1,1,1,1,1,0
A_J_Salt,1,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...
_,42,42,42,42,42,42,42,42,42,0
_gota_Krist_f,1,1,1,1,1,1,1,1,1,0
_mile_Zola,4,4,4,4,4,4,4,4,4,0


Lots of useless info there. One column should suffice

### Exercise:

- Group the dataframe by `author`. Include the following columns: `rating`, `name`, `author`. For the aggregation of the `name` column which includes the names of the books create a list with the strings containing the name of each book. Make sure that the way you aggregate the rest of the columns make sense!

- Create a new column with number of books for each author and find the most prolific author!

In [ ]:
###### Before we start : what do we do about these titles where 'name' is unreadable? Try different encodings?
auth_name = 'A_id_al_Qarni'
df[df.author == auth_name].head()

,rating,review_count,isbn,booktype,author_url,year,dir,rating_count,name,author,genres
2212,4.19,1169,,good_reads:book,https://www.goodreads.com/author/show/1201952....,2003,dir23/2750180.html,15781,ÙØ§ ØªØ­Ø²Ù,A_id_al_Qarni,None
5997,3.91,281,,good_reads:book,https://www.goodreads.com/author/show/1201952....,2006,dir60/2750008.html,3083,Ø£Ø³Ø¹Ø¯ Ø§Ù Ø±Ø£Ø© ÙÙ Ø§ÙØ¹Ø§ÙÙ,A_id_al_Qarni,None


In [ ]:
df[df.author == auth_name].iat[0,8].encode('UTF-16')

b'\xff\xfe\xd9\x00\x84\x00\xd8\x00\xa7\x00 \x00\xd8\x00\xaa\x00\xd8\x00\xad\x00\xd8\x00\xb2\x00\xd9\x00\x86\x00'

In [ ]:
# let's examine the columns we have
df.columns

Index(['rating', 'review_count', 'isbn', 'booktype', 'author_url', 'year', 'dir', 'rating_count', 'name', 'author', 'genres'], dtype='object')

Create the GroupBy table

In [ ]:
authors = df.copy()
authors = authors[['rating','name','author']].groupby('author').agg({'rating' : np.mean,
                                                                    'name' : '|'.join})

In [ ]:
authors = authors.reset_index()
authors.head()

,author,rating,name
0,A_A_Milne,4.365,Winnie-the-Pooh|The House at Pooh Corner|The H...
1,A_G_Howard,4.020,"Splintered (Splintered, #1)"
2,A_J_Cronin,4.220,The Keys of the Kingdom
3,A_J_Jacobs,3.750,The Year of Living Biblically
4,A_J_Salt,4.940,Nik Nassa & the Mark of Destiny


In [ ]:
# split the column string and make a list of string book names
authors['name'] = authors.name.str.split('|')
authors.head()

,author,rating,name
0,A_A_Milne,4.365,"[Winnie-the-Pooh, The House at Pooh Corner, Th..."
1,A_G_Howard,4.020,"[Splintered (Splintered, #1)]"
2,A_J_Cronin,4.220,[The Keys of the Kingdom]
3,A_J_Jacobs,3.750,[The Year of Living Biblically]
4,A_J_Salt,4.940,[Nik Nassa & the Mark of Destiny]


In [ ]:
# count the books - create new column
len(authors.name[0])

6

authors['num_books'] = authors['name'].str.len()
authors

In [ ]:
# sort for more prolific
authors.sort_values(by='num_books', ascending=False).iloc[0]

KeyError: 'num_books'

#### Winner is Stephen King with 56 books! OMG!!!

Perhaps you want more detailed info...

In [ ]:
dfgb_author[['rating', 'rating_count', 'review_count', 'year']].describe()

rating                                                        rating_count                                                                             review_count                                                                      year                                                                   
                       count      mean       std   min     25%    50%     75%   max        count          mean           std      min       25%      50%       75%       max        count         mean          std     min      25%     50%      75%     max count         mean         std     min      25%     50%      75%     max
author                                                                                                                                                                                                                                                                                                                                
A_A_Milne                6.0  4.365000  0.078677  4.25  4.3275  4.365  4.4025  4.48          6.0  47842.333333  57135.314010    544.0  15559.50  30547.0  50401.25  157833.0          6.0   543.000000   682.473150     1.0   219.50   309.5   499.25  1886.0   6.0  1944.166667   29.294482  1926.0  1926.25  1927.5  1952.75  1997.0
A_G_Howard               1.0  4.020000       NaN  4.02  4.0200  4.020  4.0200  4.02          1.0  17073.000000           NaN  17073.0  17073.00  17073.0  17073.00   17073.0          1.0  3194.000000          NaN  3194.0  3194.00  3194.0  3194.00  3194.0   1.0  2013.000000         NaN  2013.0  2013.00  2013.0  2013.00  2013.0
A_J_Cronin               1.0  4.220000       NaN  4.22  4.2200  4.220  4.2200  4.22          1.0   1015.000000           NaN   1015.0   1015.00   1015.0   1015.00    1015.0          1.0   109.000000          NaN   109.0   109.00   109.0   109.00   109.0   1.0  1941.000000         NaN  1941.0  1941.00  1941.0  1941.00  1941.0
A_J_Jacobs               1.0  3.750000       NaN  3.75  3.7500  3.750  3.7500  3.75          1.0  39489.000000           NaN  39489.0  39489.00  39489.0  39489.00   39489.0          1.0  4371.000000          NaN  4371.0  4371.00  4371.0  4371.00  4371.0   1.0  2007.000000         NaN  2007.0  2007.00  2007.0  2007.00  2007.0
A_J_Salt                 1.0  4.940000       NaN  4.94  4.9400  4.940  4.9400  4.94          1.0     16.000000           NaN     16.0     16.00     16.0     16.00      16.0          1.0     6.000000          NaN     6.0     6.00     6.0     6.00     6.0   1.0  2014.000000         NaN  2014.0  2014.00  2014.0  2014.00  2014.0
...                      ...       ...       ...   ...     ...    ...     ...   ...          ...           ...           ...      ...       ...      ...       ...       ...          ...          ...          ...     ...      ...     ...      ...     ...   ...          ...         ...     ...      ...     ...      ...     ...
_                       42.0  3.988095  0.301211  3.15  3.8000  4.075  4.2075  4.47         42.0  11574.642857   9368.218450     10.0   4724.00   8709.5  15572.75   39302.0         42.0  1437.214286  1216.421371     1.0   534.75  1020.5  1745.75  4785.0  42.0  1998.904762   18.721071  1941.0  1994.75  2007.0  2010.75  2014.0
_gota_Krist_f            1.0  4.340000       NaN  4.34  4.3400  4.340  4.3400  4.34          1.0   2628.000000           NaN   2628.0   2628.00   2628.0   2628.00    2628.0          1.0   243.000000          NaN   243.0   243.00   243.0   243.00   243.0   1.0  1986.000000         NaN  1986.0  1986.00  1986.0  1986.00  1986.0
_mile_Zola               4.0  3.990000  0.100995  3.84  3.9750  4.035  4.0500  4.05          4.0   8358.250000   4553.226942   3594.0   5181.75   8095.0  11271.50   13649.0          4.0   283.750000   171.069917   125.0   189.50   243.5   337.75   523.0   4.0  1652.500000  456.345264   968.0  1649.75  1878.5  1881.25  1885.0
_ric_Emmanuel_Schmitt    1.0  4.160000       NaN  4.16  4.1600  4.160  4.1600  4.16          1.0  

You can also access a `groupby` dictionary style.

In [ ]:
ratingdict = {}
for author, subset in dfgb_author:
    ratingdict[author] = (subset['rating'].mean(), subset['rating'].std())
ratingdict

{'A_A_Milne': (4.364999999999999, 0.0786765530510839),
 'A_G_Howard': (4.02, nan),
 'A_J_Cronin': (4.22, nan),
 'A_J_Jacobs': (3.75, nan),
 'A_J_Salt': (4.94, nan),
 'A_Meredith_Walters': (4.15, 0.16970562748477094),
 'A_N_Roquelaure': (3.45, 0.2262741699796951),
 'A_S_Byatt': (3.86, nan),
 'A_S_King': (3.93, nan),
 'A_id_al_Qarni': (4.050000000000001, 0.19798989873223347),
 'Abbi_Glines': (4.179285714285713, 0.12916681436834276),
 'Abdul_Rahman_Munif': (4.03, nan),
 'Abigail_Gibbs': (3.82, nan),
 'Abigail_Roux': (4.47, 0.22876479915697992),
 'Abigail_Thomas': (3.68, nan),
 'Abolqasem_Ferdowsi': (4.52, nan),
 'Abraham_Verghese': (4.26, nan),
 'Abul_Hasan_Ali_Nadwi': (4.15, nan),
 'Adam_Hochschild': (4.14, nan),
 'Adam_Johnson': (4.03, nan),
 'Adam_Levin': (4.04, nan),
 'Adam_Rex': (4.14, nan),
 'Adam_Smith': (3.82, nan),
 'Addison_Moore': (3.78, nan),
 'Adeline_Yen_Mah': (4.01, nan),
 'Adolf_Hitler': (2.97, nan),
 'Adolfo_Bioy_Casares': (4.06, nan),
 'Aeschylus': (3.96, nan),
 'Aesop':

<div class="exercise"><b>Exercise</b></div>

Lets get the best-rated book(s) for every year in our dataframe.

In [ ]:
#Using .groupby, we can divide the dataframe into subsets by the values of 'year'.
#We can then iterate over these subsets
# your code here

subset_year=df.groupby('year')
for i in subset_year:

    maxRating=i[1].rating.max()
    bestRatedBook = i[1][i[1].rating == maxRating]
    if bestRatedBook.shape[0] > 1:
        print(i[0], bestRatedBook.name.values, bestRatedBook.rating.values)
    else:
        print(i[0], bestRatedBook.name.values[0], bestRatedBook.rating.values[0])

-1500 The Epic of Gilgamesh 3.6
-800 The Iliad/The Odyssey 4.01
-560 Aesop's Fables 4.03
-512 The Art of War 3.92
-500 The Bhagavad Gita 4.06
-458 The Oresteia 3.96
-442 Antigone 3.52
-440 The Histories 3.94
-431 Medea 3.8
-429 Oedipus Rex 3.64
-411 The History of the Peloponnesian War 3.84
-400 Complete Works 4.3
-390 Apology 4.11
-380 The Republic 3.85
-370 The Symposium 3.99
-360 Five Dialogues 4.07
-350 The Nicomachean Ethics 3.88
-29 The Aeneid 3.77
4 The Seven Spiritual Laws of Success 4.04
8 Metamorphoses 4.01
13 The Day Jesus Rode Into Croydon 3.85
100 The New Oxford Annotated Bible, New Revised Standard Version 4.3
121 The Twelve Caesars 4.02
130 Holy Bible 4.45
158 The Golden Ass 3.88
174 Meditations 4.16
200 The Garden of Eden 3.75
201 Fe Fi FOE Comes 4.37
213 NaÃ¯ve. Super 3.9
397 Confessions 3.82
632 The Quran 4.37
800 Beowulf 3.38
900 The Arabian Nights 4.04
968 La BÃªte humaine (Les Rougon-Macquart, #17) 4.02
1002 The Pillow Book 4.04
1008 The Tale of Genji 3.68
1010 Sha

In [ ]:
print(bestbook)

      rating  review_count isbn         booktype                                         author_url  year                                             dir  rating_count                                  name                      author genres
1717     5.0            28       good_reads:book  https://www.goodreads.com/author/show/6467808....  2014  dir18/22204746-an-elephant-is-on-my-house.html            64            An Elephant Is On My House  Othen_Donald_Dale_Cummings   None
5563     5.0             9       good_reads:book  https://www.goodreads.com/author/show/7738947....  2014                   dir56/21902777-untainted.html            14  Untainted (Photographer Trilogy, #3)              Sarah_Robinson   None
